# Ejercicio Ciudades

El presente notebook presenta el analisis del desafio de ciudades. El tiempo es de 3 horas. A continuación se presentan el proceso de los resultados. Este notebook es reproducible en binder usando el link del repositorio.



El objetivo es determinar si los estados de la costa este tienen más o menos ciudades que comparten nombre con el Reino Unido (UK), para ello se dispone de una base de datos pública.

Intentaré explicar paso a paso lo que voy haciendo incluyendo analisis exploratorios, etc.

In [1]:
import datetime as dt
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [2]:
ti=dt.datetime.now() # tiempo inicial

En primer lugar abrimos la base de datos y realizamos un poco de exploración

In [5]:
url = "https://public.opendatasoft.com/explore/dataset/worldcitiespop/download/?format=geojson&disjunctive.country=true&refine.country=us&refine.country=gb&timezone=America/Santiago&lang=en"
geo = gpd.read_file(url)

In [6]:
geo

,city,country,region,longitude,latitude,accentcity,population,geometry
0,lost creek,us,KY,-83.325278,37.481667,Lost Creek,NaN,POINT (-83.32528 37.48167)
1,edgewood village,us,LA,-93.842500,32.400556,Edgewood Village,NaN,POINT (-93.84250 32.40056)
2,trout dale,us,VA,-81.440556,36.701944,Trout Dale,NaN,POINT (-81.44056 36.70194)
3,trivoli,us,IL,-89.891944,40.690556,Trivoli,NaN,POINT (-89.89194 40.69056)
4,barhamsville,us,VA,-76.840556,37.454444,Barhamsville,NaN,POINT (-76.84056 37.45444)
...,...,...,...,...,...,...,...,...
159685,bridger view trailer court,us,MT,-111.020833,45.702778,Bridger View Trailer Court,NaN,POINT (-111.02083 45.70278)
159686,rising city,us,NE,-97.295833,41.199444,Rising City,NaN,POINT (-97.29583 41.19944)
159687,northwood,us,MD,-76.904167,38.647222,Northwood,NaN,POINT (-76.90417 38.64722)
159688,moss bluff,us,LA,-93.190556,30.302500,Moss Bluff,NaN,POINT (-93.19056 30.30250)


Una vez importados los datos a formato geopandas (que servirán para graficar el resultado), usamos pandas para importar los datos como tabla.

In [9]:
url = "https://public.opendatasoft.com/explore/dataset/worldcitiespop/download/?format=csv&disjunctive.country=true&refine.country=us&refine.country=gb&timezone=America/Santiago&lang=en&use_labels_for_header=true&csv_separator=%3B"
df = pd.read_csv(url,sep=';')


In [10]:
df.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,geopoint
0,us,lost creek,Lost Creek,KY,NaN,37.481667,-83.325278,"37.4816667,-83.3252778"
1,us,edgewood village,Edgewood Village,LA,NaN,32.400556,-93.842500,"32.4005556,-93.8425"
2,us,trout dale,Trout Dale,VA,NaN,36.701944,-81.440556,"36.7019444,-81.4405556"
3,us,trivoli,Trivoli,IL,NaN,40.690556,-89.891944,"40.6905556,-89.8919444"
4,us,barhamsville,Barhamsville,VA,NaN,37.454444,-76.840556,"37.4544444,-76.8405556"


Una ves cargada ambas bases de datos nos hace falta una pieza de información más: Qué estados se consideran parte de cada costa. Usando google. [https://en.wikipedia.org/wiki/East_Coast_of_the_United_States#:~:text=The%2014%20states%20that%20have,Carolina%2C%20Georgia%2C%20and%20Florida.]

Usando esto tenemos el listado de la costa este: **Maine, New Hampshire, Massachusetts, Rhode Island, Connecticut, New York, New Jersey, Delaware, Maryland, Virginia, North Carolina, South Carolina, Georgia, y Florida.** Es importante convertir esto a los codigos de 2 letras para poder filtrar luego.

[https://en.wikipedia.org/wiki/West_Coast_of_the_United_States] nos indica que generalmente se consideran **California, Oregon, Washington, y Alaska**

In [ ]:
#creamos una lista con las costas, por sus codigos segun https://www.infoplease.com/us/postal-information/state-abbreviations-and-state-postal-codes
ec=['ME','NH','MA','RI','CT','NY','NJ','DE','MD','VA','NC','SC','GA','FL']
wc=['CA','OR','WA','AK']
# procedemos a filtrar la lista para incluir solo ciudades en cada área.
cec=df[df['Region'].isin(ec) & df['Country']=='us']
cec
cwc=df[df['Region'].isin(wc) & df['Country']=='us']

# finalmente recuperamos los nombres de ciudades del reino unido para comparar
uk=df.loc[df['Country']=='gb','City']

Antes de realizar la comparación y contar el numero de ciudades, me parece que aquelas ciudades de EEUU que tengan el prefijo "new" debiesen contar. Creo que el nombre "New" York esta claramente asociado a la ciudad de "York", por lo que antes de realizar las comparaciones, quitaré todos los prefijos "new " de los nombres de las ciudades. Es importante considerar el espacio antes para evitar cambiar el nombre de ciudades como "newark"



In [ ]:
cec["City2"]=cec.City.str.replace("new ","")
cwc["City2"]=cwc.City.str.replace("new ","")

Procedemos a buscar cuantas ciudades de la costa este y oeste estan en ambos casos.

In [ ]:
matche=cec["City2"].isin(uk.values)
matchw=cwc["City2"].isin(uk.values)
print(len(matche))
print(len(matchw))

Obtenidos los numeros absolutos, ahora intentaremos representarlos de una forma más amigable. Para esto utilizaremos las capacidades interactivas de este notebook.

In [ ]:
tf-ti

In [13]:
tf=dt.datetime.now() 


datetime.timedelta(seconds=24, microseconds=609288)